<a href="https://colab.research.google.com/github/BigTMiami/AdaptOrDie/blob/main/TAPT_base_pretrain_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Summary
This trains a roberta-base model on 100 epochs of TAPT helpfulness

## Base Model
* No Adapter

## Early stopping
* early_stopping_patience=5

## Pytorch compile
* torch_compile=True




# Setup

In [1]:
from time import time
start_time = time()

In [2]:
# from google.colab import drive
# drive.mount("/content/drive")
# %cd '/content/drive/MyDrive/AdaptOrDie'

In [3]:
!pip install datasets
!pip install transformers[torch]
# !pip install adapters
# !pip install scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 6.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86

In [4]:
!pip install pynvml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 2.1 MB/s eta 0:00:00


In [5]:
from pynvml import *

def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    gpu_used = info.used//1024**2
    print(f"GPU {gpu_used} MB")

print_gpu_utilization()

GPU 448 MB


In [6]:
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    # Calculate accuracy
    accuracy = accuracy_score(labels, preds)

   # Calculate precision, recall, and F1-score
    f1 = f1_score(labels, preds, average='macro')

    return {
        'accuracy': accuracy,
        'f1_macro': f1
    }

# Pretraining

In [7]:
from datasets import load_dataset

pretrain_dataset_name = "BigTMiami/amazon_helpfulness_TAPT_pretraining_dataset"

pretrain_train_dataset, pretrain_eval_dataset = load_dataset(pretrain_dataset_name, split=["test","dev"])

print(pretrain_train_dataset)
print(pretrain_eval_dataset)

Generating train split:   0%|          | 0/44876 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/9754 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/1980 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 9754
})
Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 1980
})


In [8]:
from transformers import AutoConfig, AutoModelForMaskedLM
pretrain_model = AutoModelForMaskedLM.from_pretrained("roberta-base")

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

In [9]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("roberta-base")

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [10]:
from transformers import DataCollatorForLanguageModeling

pretraining_data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm_probability=0.15
)

In [11]:
from transformers import TrainingArguments

pretraining_args = TrainingArguments(
    output_dir="tapt_helpfulness_base_pretraining_model_final",
    learning_rate=0.0005, # Paper for DAPT training
    per_device_train_batch_size=32, # 346,m - TRYING SMALLER BATCH SIZE
    per_device_eval_batch_size=32, #346,m - TRYING SMALLER BATCH SIZE
    gradient_accumulation_steps=11, # Creates effective batch of 32*11*6 = 2112
    num_train_epochs=100, # Just one epoch for one pass on data
    weight_decay=0.01,
    warmup_ratio=0.06, # Paper: warmup proportion of 0.06
    adam_epsilon=1e-6, # Paper 1e-6 (huggingface default 1e-08)
    adam_beta1=0.9, # Paper: Adam weights 0.9
    adam_beta2=0.98, # Paper: Adam weights 0.98 (huggingface default  0.999)
    lr_scheduler_type="linear",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2, # Saves latest 2 checkpoints
    torch_compile=True,  # Much Faster
    push_to_hub=True,
    hub_strategy="checkpoint", # Only pushes at end with save_model()
    logging_strategy="epoch", # Is default
    metric_for_best_model="eval_loss", # Is default
    load_best_model_at_end=True,
    report_to="tensorboard",
)

In [12]:
from transformers import EarlyStoppingCallback

early_stopping_callback = EarlyStoppingCallback(early_stopping_patience=5)

In [13]:
from transformers import Trainer

pretrain_trainer = Trainer(
    model=pretrain_model,
    args=pretraining_args,
    train_dataset=pretrain_train_dataset,
    eval_dataset=pretrain_eval_dataset,
    data_collator=pretraining_data_collator,
    callbacks=[early_stopping_callback],
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [14]:
pretrain_results = pretrain_trainer.train()
print(pretrain_results)

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Epoch,Training Loss,Validation Loss
0,1.930000,1.612109
1,1.668300,1.553462
2,1.625200,1.525793
3,1.665100,1.542448
4,1.608500,1.571555
5,1.607800,1.571021
6,1.615800,1.580652
7,1.649100,1.584779


There were missing keys in the checkpoint model loaded: ['lm_head.decoder.weight', 'lm_head.decoder.bias'].


TrainOutput(global_step=221, training_loss=1.6701751639940081, metrics={'train_runtime': 990.2471, 'train_samples_per_second': 985.007, 'train_steps_per_second': 2.727, 'total_flos': 2.054313063191347e+16, 'train_loss': 1.6701751639940081, 'epoch': 7.97})


In [15]:
import pandas as pd

pd.DataFrame(pretrain_trainer.state.log_history)

,loss,grad_norm,learning_rate,epoch,step,eval_loss,eval_runtime,eval_samples_per_second,eval_steps_per_second,train_runtime,train_samples_per_second,train_steps_per_second,total_flos,train_loss
0,1.9300,2.078114,0.000083,0.97,27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,0.97,27,1.612109,9.2218,214.709,6.723,NaN,NaN,NaN,NaN,NaN
2,1.6683,1.837802,0.000170,1.98,55,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,1.98,55,1.553462,8.7032,227.502,7.124,NaN,NaN,NaN,NaN,NaN
4,1.6252,1.844487,0.000256,2.99,83,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,2.99,83,1.525793,8.7055,227.442,7.122,NaN,NaN,NaN,NaN,NaN
6,1.6651,1.817854,0.000340,3.97,110,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,3.97,110,1.542448,8.6763,228.209,7.146,NaN,NaN,NaN,NaN,NaN
8,1.6085,2.610339,0.000426,4.98,138,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,4.98,138,1.571555,8.7093,227.342,7.119,NaN,NaN,NaN,NaN,NaN


In [16]:
after_pretrain_eval_results = pretrain_trainer.evaluate()
after_pretrain_eval_results_loss = after_pretrain_eval_results["eval_loss"]
print(f"After pretraining loss: {after_pretrain_eval_results_loss:.4f}")
print(after_pretrain_eval_results)

After pretraining loss: 1.5492
{'eval_loss': 1.5492362976074219, 'eval_runtime': 8.7487, 'eval_samples_per_second': 226.321, 'eval_steps_per_second': 7.087, 'epoch': 7.97}


In [17]:
pretrain_trainer.push_to_hub()

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

events.out.tfevents.1713222126.178b6c650117.245.0:   0%|          | 0.00/8.84k [00:00<?, ?B/s]

events.out.tfevents.1713223158.178b6c650117.245.1:   0%|          | 0.00/359 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/BigTMiami/tapt_helpfulness_base_pretraining_model_final/commit/4d4e5e090a5cc7b358fe0ceb7f60fc333828ade4', commit_message='End of training', commit_description='', oid='4d4e5e090a5cc7b358fe0ceb7f60fc333828ade4', pr_url=None, pr_revision=None, pr_num=None)

In [18]:
end_time = time()
total_time = end_time - start_time
print(f"Total Time: {total_time/60:.0f} minutes OR {total_time:.0f} seconds")

Total Time: 19 minutes OR 1161 seconds


# Classification

In [19]:
print("Disconnecting Session")
from google.colab import runtime
runtime.unassign()

Disconnecting Session
